# Worlds Bank data ETL

This data source is downloaded from the following webpage: [World Bank Health, Nutrition, and Population Statistics](https://databank.worldbank.org/source/health-nutrition-and-population-statistics/Type/TABLE/preview/on#) into an Excel file. The main objective of this project is to display only the required information for the selected country and desired series, then transform and store it in a SQL database so that it can be accessed by multiple users in the future. The database will include the names of the variables that compose it. This ETL process simplifies the cleaning and storage of the desired information in a specific format within a data warehouse, making it easier for future consultations.

## Extract
Extract information and store it as a dataframe

In [15]:
import pandas as pd
from sqlalchemy import create_engine

# Path to the Excel file
folder_path = r"C:\Users\Juan Esteban\Desktop\Proyectos Python\04. ETL"
import os

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(r"C:\Users\Juan Esteban\Desktop\Proyectos Python\04. ETL\Population_Statistics.xlsx", sheet_name='Data')

# Data preview: Use these commands to explore the data.
# df.columns # Name of the columns
# df['Series Name'].unique() # Unique information in the 'Series Name' column


## Transform
Transform information to make it easier to analyze: In this case, we are separating the information according to its categories.

In [16]:
# Define the texts you want to filter
filter_series_name = ['Unemployment, total (% of total labor force)']

# Define the countries you want to filter in 'Country Name'
filter_country_name = ['Argentina', 'Armenia']

# Apply both filters
df2 = df[(df['Series Name'].isin(filter_series_name)) & (df['Country Name'].isin(filter_country_name))]

# Preview the data frame
df2

,Series Name,Series Code,Country Name,Country Code,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
224,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Argentina,ARG,7.268,7.58,8.083,8.347,9.221,9.843,11.461,8.736,6.805,6.178
225,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Armenia,ARM,12.201,12.468,12.732,12.968,13.212,12.2,12.18,10.012,8.608,8.586


Exclude unecesary columns

In [27]:
# Chose the columns to exclude
columns_to_exclude = ['Series Code','Country Code']

# Create a new DataFrame excluding those columns
df3 = df2.drop(columns=columns_to_exclude)


['Series Code', 'Country Code']

Pivot the data to make it easier to analyze.

In [14]:
# Pivot the year columns
df_pivoted = pd.melt(df3, id_vars=['Series Name', 'Country Name'], 
                     var_name='Year', value_name='Value')

# Display the pivoted DataFrame
df_pivoted


,Series Name,Country Name,Year,Value
0,"Unemployment, total (% of total labor force)",Argentina,2014 [YR2014],7.268
1,"Unemployment, total (% of total labor force)",Armenia,2014 [YR2014],12.201
2,"Unemployment, total (% of total labor force)",Argentina,2015 [YR2015],7.58
3,"Unemployment, total (% of total labor force)",Armenia,2015 [YR2015],12.468
4,"Unemployment, total (% of total labor force)",Argentina,2016 [YR2016],8.083
5,"Unemployment, total (% of total labor force)",Armenia,2016 [YR2016],12.732
6,"Unemployment, total (% of total labor force)",Argentina,2017 [YR2017],8.347
7,"Unemployment, total (% of total labor force)",Armenia,2017 [YR2017],12.968
8,"Unemployment, total (% of total labor force)",Argentina,2018 [YR2018],9.221
9,"Unemployment, total (% of total labor force)",Armenia,2018 [YR2018],13.212


## Load
Craete a new excle file and load the data into the excel

Get the current directory where the code is being executed

In [21]:
import os
import xlwings as xw

current_directory = os.getcwd()

Define the path for the new file:

In [30]:
# Define the path for the new Excel file
new_excel_path = os.path.join(current_directory, 'NewFile.xlsx')


Create the new Excel file:

In [31]:
# Create a new Excel workbook
plantilla_excel = xw.Book()

resumen= plantilla_excel.sheets['Hoja1']

**Insert the DataFrame Information into the Excel File**


In [32]:
# Assuming 'data' is a DataFrame, get the list of column names
columnas = df_pivoted.columns.tolist()

# Write the column names to cell B2 in the worksheet
resumen.range('A1').value = columnas

# Write the DataFrame values starting from cell B3 in the worksheet
resumen.range('A2').value = df_pivoted.values

Save the file with a descriptive name that reflects the information contained in the file.

In [35]:
# Format the filters into strings
series_name_str = ', '.join(filter_series_name)
country_name_str = ', '.join(filter_country_name)

# Create the filename with the filters included
filename = f"Series_{series_name_str}_Countries_{country_name_str}.xlsx"

# Save the Excel file
plantilla_excel.save(filename)

# Cerrar el archivo Excel
plantilla_excel.close()